In [18]:
# pull down A1, A2 readnames
import pysam
a1_names = {x.qname for x in pysam.Samfile('A1.binreads.bam')}
a2_names = {x.qname for x in pysam.Samfile('A2.binreads.bam')}

In [25]:
# use these names to subset the original FASTQs producing correctly paired reads
# also keep the barcodes
tagged_fq = '/hive/users/ifiddes/notch2nl_berkeley_data/E2del19N_E2del68_combined_longranger/E2del68_E2del19N_combined/E2del68_E2del19N_combined.sorted.fastq'
from collections import *
bcode_map = {}
a1_reads = defaultdict(list)
a2_reads = defaultdict(list)
import sys
sys.path.append('/hive/users/ifiddes/comparativeAnnotator/')
from sonLib.bioio import *
for name, seq, qual in fastqRead(tagged_fq):
    base_name, bcode = name.split()
    base_name = base_name[:-2]
    bcode_map[base_name] = bcode
    if base_name in a1_names:
        a1_reads[base_name].append([name, seq, qual])
    elif base_name in a2_names:
        a2_reads[base_name].append([name, seq, qual])

In [29]:
# construct a fastq for each
with open('A1.fastq', 'w') as outf:
    for p in a1_reads.itervalues():
        if len(p) != 2:
            continue
        for name, seq, qual in p:
            fastqWrite(outf, name, seq, qual)
with open('A2.fastq', 'w') as outf:
    for p in a2_reads.itervalues():
        if len(p) != 2:
            continue
        for name, seq, qual in p:
            fastqWrite(outf, name, seq, qual)

In [30]:
consensus = '/hive/users/ifiddes/notch2nl_berkeley_data/new_consensus/extended_alignment_consensus.fa'
!cat A1.fastq A2.fastq > A_combined.fastq
!bwa mem -t 8 -p {consensus} A_combined.fastq | samtools view -b - | sambamba sort -o A_consensus_mapped.bam /dev/stdin

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 573478 sequences (80000181 bp)...
[M::process] 0 single-end sequences; 573478 paired-end sequences
[M::process] read 207504 sequences (28946808 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (2927, 135092, 99916, 2672)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (181, 298, 493)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 1117)
[M::mem_pestat] mean and std.dev: (356.25, 244.52)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 1429)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (272, 371, 495)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 941)
[M::mem_pestat] mean and std.dev: (376.05, 193.48)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 1164)
[M::mem_pestat] analyzing insert

In [35]:
# re-add the BX tags
sh = pysam.Samfile('A_consensus_mapped.bam')
with pysam.Samfile('A_consensus_mapped.tagged.bam', 'wb', template=sh) as outf:
    for x in sh:
        name = x.qname
        if name in a1_names:
            truth = '0'
        elif name in a2_names:
            truth = '1'
        else:
            assert False
        bcode = bcode_map[name]
        x.set_tag('TR', truth)
        x.set_tag('BX', bcode)
        outf.write(x)